In [1]:
import pandas as pd


In [2]:
import numpy as np 

def linParts(numAtoms, numThreads):
    parts = np.linspace(0,numAtoms, min(numThreads, numAtoms)+1)
    parts = np.ceil(parts).astype(int)
    return parts

In [3]:
def featImpMDI(fit, featNames):
    #feature importance based on in sample mean impurity reduction 
    df0 = {i:tree.feature_importances_ for i, tree in enumerate(fit.estimators_)}
    df0 = pd.DataFrame.from_dict(df0, orient='index')
    df0.columns = featNames
    df0 = df0.replace(0, np.nan) #because max features=1, any 0 is a non randomly selected feature
    imp = pd.concat({'mean':df0.mean(), 'std':df0.std()*df0.shape[0]**-.5}, axis=1)
    imp/=imp['mean'].sum()
    return imp

In [4]:
def featImpMDA(clf, X, y, cv, sample_weight, t1, pctEmbargo, scoring="neg_log_loss"):
    #feature importance based on outside of sample score reduction
    if scoring not in ['neg_log_loss', 'accuracy']:
        raise Exception('wrong scoring method')
    from sklearn.metrics import log_loss, accuracy_score
    cvGen = PurgedKFold(n_splits=cv, t1=t1, pctEmbargo=pctEmbargo)
    scr0, scr1 = pd.Series(), pd.DataFrame(columns=X.columns)
    for i, (train, test) in enumerate(cvGen.split(X=X)):
        X0, y0, w0 = X.iloc[train,:], y.iloc[train], sample_weight.iloc[train]
        X1, y1, w1 = X.iloc[test, :], y.iloc[test], sample_weight.iloc[train]
        fit = clf.fit(X=X0, y=y0, sample_weight=w0.values)
        if scoring=='neg_log_loss':
            prob = fit.predict_proba(X1)
            scr0.loc[i]= -log_loss(y1,prob,sample_weight=w1.values,labels=clf.classes_)
        else:
            pred = fit.predict(X1)
            scr0.loc[i]=accuracy_score(y1,pred)
        for j in X.columns:
            X1_ = X1.copy(deep=True)
            np.random.shuffle(X1_[j].values) #permutation of a single column
            if scoring=='neg_log_loss':
                prob = fit.predict_proba(X1_)
                scr1.loc[i,j]=-log_loss(y1, prob, sample_weight=w1.values, labels=clf.classes_)
            else: 
                pred=fit.predict(X1_)
                scr1.loc[i,j]=accuracy_score(y1, pred)
    imp = (0-scr1).add(scr0, axis=0)
    if scoring=='neg_log_loss':
        imp = imp/-scr1
    else: 
        imp = imp/(1.-scr1)
    imp = pd.concat({'mean':imp.mean(), 'std':imp.std()*imp.shape[0]**-.5}, axis=1)
    return imp, scr0.mean()

In [5]:
#single feature importance (OOS) prevents discarding important features just because they are important. 
#However it loses any joint effects or hierachical importance
def auxFeatImpSFI(featNames, clf, trnsX, cont, scoring, cvGen):
    imp = pd.DataFrame(columns=['mean', 'std'])
    for featName in featNames:
        df0 = cvScore(clf, X=trnsX[[featName]], y=cont['bin'], sample_weight=cont['w'], scoring=scoring, cvGen=cvGen)
        imp.loc[featName,'mean']=df0.mean()
        imp.loc[featName, 'std']=df0.std()*df0.shape[0]**-.5
    return imp

In [6]:
def get_eVec(dot, varThres):
    #compute eigenvector from dot product matrix, reduce dimension
    eVal, eVec = np.linalg.eigh(dot)
    idx = eVal.argsort()[::-1] #arguments for sorting eVal desc
    eVal, eVec = eVal[idx], eVec[:,idx]
    # only positive eVals
    eVal = pd.Series(eVal, index=['PC_'+str(i+1) for i in range(eVal.shape[0])])
    eVec = pd.DataFrame(eVec, index=dot.index, columns=eVal.index)
    eVec = eVec.loc[:, eVal.index]
    # reduce dimension, form PCs
    cumVar = eVal.cumsum()/eVal.sum()
    dim = cumVar.values.searchsorted(varThres)
    eVal, eVec = eVal.iloc[:dim+1], eVec.iloc[:, :dim+1]
    return eVal, eVec

def orthoFeats(dfX, varThres=.95):
    #given a dataframe dfX of features, compute orthofeatures dfP
    dfZ = dfX.sub(dfX.mean(), axis=1).div(dfX.std(), axis=1) #standardize
    dot = pd.DataFrame(np.dot(dfZ.T,dfZ), index=dfX.columns, columns=dfX.columns)
    eVal, eVec = get_eVec(dot, varThres)
    dfP = np.dot(dfZ, eVec)
    return dfP

In [7]:
#compute weighted kendall's tau betwen feature importance and inverse PCA ranking (synthetic data for example)
import numpy as np
from scipy.stats import weightedtau
featImp = np.array([.55,.33,.07,.05])
pcRank = np.array([1,2,4,3])
weightedtau(featImp,pcRank**-1.)[0]

0.8133333333333331

In [8]:
from sklearn.model_selection._split import _BaseKFold
class PurgedKFold(_BaseKFold):
    '''
    Extend Kfold to work with labels that span intervals
    the train is purged of observations overlapping test-label intervals
    test set is assumed contiguous (shuffle=False), w/o training examples in between'''
    def __init__(self, n_splits=3, t1=None, pctEmbargo=0.):
        if not isinstance(t1, pd.Series):
            raise ValueError('must be pd series')
        super(PurgedKFold, self).__init__(n_splits, shuffle=False, random_state=None)
        self.t1=t1
        self.pctEmbargo=pctEmbargo
    def split(self, X, y=None, groups=None):
        if(X.index==self.t1.index).sum()!=len(self.t1):
            raise ValueError('X and thruDateValues must have the same index')
        indices = np.arange(X.shape[0])
        mbrg = int(X.shape[0]*self.pctEmbargo)
        test_starts=[(i[0], i[-1]+1) for i in np.array_split(np.arange(X.shape[0]), self.n_splits)]
        for i,j in test_starts:
            t0 = self.t1.index[i] #start of test set
            test_indices = indices[i:j]
            maxT1Idx = self.t1.index.searchsorted(self.t1[test_indices].max())
            train_indices = self.t1.index.searchsorted(self.t1[self.t1<=t0].index)
            train_indices = np.concatenate((train_indices, indices[maxT1Idx+mbrg:]))
            yield train_indices, test_indices

In [9]:
# creating a synthetic dataset of 40 features: 10 of which are informative, 10 are redundant and 20 are noise. 10000 observations
def getTestData(n_features=40,n_informative=10, n_redundant=10, n_samples=10000):
    from sklearn.datasets import make_classification
    trnsX, cont = make_classification(n_samples=n_samples, n_features=n_features, n_informative=n_informative, n_redundant=n_redundant, random_state=0,shuffle=False)
    df0 = pd.DatetimeIndex(periods=n_samples,freq=pd.tseries.offsets.BDay(), end=pd.datetime.today())
    trnsX, cont = pd.DataFrame(trnsX,index=df0), pd.Series(cont,index=df0).to_frame('bin')
    df0 = ['I'+str(i) for i in range(n_informative)]+['R_'+str(i) for i in range(n_redundant)]
    df0+=['N_'+str(i) for i in range(n_features-len(df0))]
    trnsX.columns=df0
    cont['w']=1./cont.shape[0]
    cont['t1']=pd.Series(cont.index, index=cont.index)
    return trnsX, cont

In [10]:
def featImportance(trnsX, cont, n_estimators=1000, cv=100, max_samples=1., numThreads=24, pctEmbargo=0, scoring='accuracy', method='SFI', minWLeaf=0., **kargs):
    #feature importance from a random forest
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import BaggingClassifier
    
    n_jobs=(-1 if numThreads>1 else 1)
    #prepare classifier, cv. max_features=1 to prevent masking
    clf = DecisionTreeClassifier(criterion='entropy', max_features=1,class_weight='balanced', min_weight_fraction_leaf=minWLeaf)
    clf = BaggingClassifier(base_estimator=clf,n_estimators=n_estimators,max_features=1.,max_samples=max_samples, oob_score=True, n_jobs=n_jobs)
    
    fit=clf.fit(X=trnsX, y=cont['bin'], sample_weight=cont['w'].values)
    oob=fit.oob_score_
    if method=='MDI':
        imp=featImpMDI(fit,featNames=trnsX.columns)
        oos=cvScore(clf, X=trnsX, y=cont['bin'], cv=cv,sample_weight=cont['w'],t1=cont['t1'],pctEmbargo=pctEmbargo, scoring=scoring).mean()
    elif method=='MDA':
        imp, oos=featImpMDA(clf,X=trnsX,y=cont['bin'], cv=cv, sample_weight=cont['w'],t1=cont['t1'], pctEmbargo=pctEmbargo, scoring=scoring)
    elif method=='SFI':
        cvGen=PurgedKFold(n_splits=cv,t1=cont['t1'], pctEmbargo = pctEmbargo)
        oos = cvScore(clf, X=trnsX, y=cont['bin'], sample_weight=cont['w'], scoring=scoring, cvGen=cvGen).mean()
        clf.n_jobs=1
        imp=mpPandasObj(auxFeatImpSFI,('featNames',trnsX.columns), numThreads, clf=clf, trnsX=trnsX,cont=cont, scoring=scoring, cvGen=cvGen)
    return imp, oob, oos

In [11]:
def testFunc(n_features=40, n_informative=10, n_redundant=10, n_estimators=1000, n_samples=10000, cv=10):
    trnsX, cont = getTestData(n_features, n_informative, n_redundant, n_samples)
    dict0 = {'minWLeaf':[0.], 'scoring':['accuracy'],'method':['MDI','MDA','SFI'], 'max_samples':[1.]}
    jobs, out = (dict(izip(dict0,i)) for i in product(*dict0.values())),[]
    kargs = {'pathOut':'./testFunc/', 'n_estimators':n_estimators, 'tag':'testFunc', 'cv':cv}
    for job in jobs:
        job['simNum']=job['method']+'_'+job['scoring']+'_'+'%.2f'%job['minWLeaf']+'_'+str(job['max_samples'])
        print(job['simNum'])
        kargs.update(job)
        imp, oob, oos = featImportance(trnsX=trnsX,cont=cont, **kargs)
        plotFeatImportance(imp=imp, oob=oob,oos=oos, **kargs)
        df0 = imp[['mean']]/imp['mean'].abs().sum()
        df0['type']=[i[0] for i in df0.index]
        df0 = df0.groupby('type')['mean'].sum().to_dict()
        df0.update({'oob':oob,'oos':oos});df0.update(job)
        out.append(df0)
    out = pd.DataFrame(out).sort_values(['method','scoring','minWLeaf','max_samples'])
    out = out['method','scoring','minWLeaf','max_samples','I','R','N','oob','oos']
    out.to_csv(kargs['pathOut']+'stats.csv')
    return


In [12]:
from itertools import product
izip = zip

def mpPandasObj(func,pdObj,numThreads=24,mpBatches=1,linMols=True, **kargs):
    import pandas as pd
    if linMols:
        parts=linParts(len(pdObj[1]),numThreads*mpBatches)
    else:
        parts=nestedParts(len(pdObj[1]),numThreads*mpBatches)
    jobs=[]
    for i in range(1, len(parts)):
        job={pdObj[0]:pdObj[1][parts[i-1]:parts[i]],'func':func}
        job.update(kargs)
        jobs.append(job)
    if numThreads==1:
        out=processJobs_(jobs)
    else:
        out=processJobs_(jobs)
    if isinstance(out[0], pd.DataFrame):
        df0=pd.DataFrame()
    elif isinstance(out[0],pd.Series):
        df0=pd.Series()
    else: 
        return out
    for i in out:
        df0=df0.append(i)
    return df0.sort_index()

import multiprocessing as mp
import time
import datetime as dt
import sys
import matplotlib.pyplot as mpl
def expandCall(kargs):
    #expand the arguments of a callback function 
    func = kargs['func']
    del kargs['func']
    out = func(**kargs)
    return out

def processJobs_(jobs):
    #run jobs sequentially. for debugging
    out=[]
    for job in jobs:
        out_=expandCall(job)
        out.append(out_)
    return out

def cvScore(clf, X, y, sample_weight, scoring='neg_log_loss', t1=None, cv=None, cvGen=None,pctEmbargo=None):
    if scoring not in ['neg_log_loss','accuracy']:
        raise Exception('wrong scoring method')
    from sklearn.metrics import log_loss, accuracy_score
    if cvGen is None:
        cvGen = PurgedKFold(n_splits=cv, t1=t1, pctEmbargo=pctEmbargo) #purged
    scores=[]
    for train, test in cvGen.split(X=X):
        fit = clf.fit(X=X.iloc[train, :], y=y.iloc[train],sample_weight=sample_weight.iloc[train].values)
        if scoring == 'neg_log_loss':
            prob=fit.predict_proba(X.iloc[test, :])
            score_=-log_loss(y.iloc[test],prob,sample_weight=sample_weight.iloc[test].values, labels = clf.classes_)
        else: 
            pred = fit.predict(X.iloc[test,:])
            score_ = accuracy_score(y.iloc[test],pred,sample_weight=sample_weight.iloc[test].values)
        scores.append(score_)
    return np.array(scores)

In [13]:
def plotFeatImportance(pathOut,imp,oob,oos,method,tag=0,simNum=0,**kargs):
    mpl.Figure(figsize=(10,imp.shape[0]/5.))
    imp = imp.sort_values('mean', ascending=True)
    ax = imp['mean'].plot(kind='barh',color='b',alpha=.25,xerr=imp['std'],error_kw={'ecolor':'r'})
    if method=='MDI':
        mpl.xlim([0,imp.sum(axis=1).max()])
        mpl.axvline(1./imp.shape[0],linewidth=1,color='r',linestyle='dotted')
    ax.get_yaxis().set_visible(False)
    for i, j in zip(ax.patches, imp.index):
        ax.text(i.get_width()/2,i.get_y()+i.get_height()/2,j,ha='center',va='center',color='black')
    mpl.title('tag='+tag+' | simNum='+str(simNum)+' | oob='+str(round(oob,4))+' | oos'+str(round(oos,4)))
    mpl.savefig(pathOut+'featImportance_'+str(simNum)+'.png',dpi=100)
    mpl.clf();mpl.close()
    return

In [14]:
testFunc()

MDI_accuracy_0.00_1.0


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


MDA_accuracy_0.00_1.0
SFI_accuracy_0.00_1.0


KeyboardInterrupt: 